In [2]:
import numpy as np #for numerical computations
import pandas as pd #for dataframe operations

from matplotlib import pyplot as plt #for viewing images and plots
%matplotlib inline 
#So that Matplotlib plots don't open in separate windows outside the notebook

import urllib #For fetching data from Web URLs

import cv2   #For image processing

from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of dataset

#All tensorflow utilities for creating, training and working with a CNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adamax
import tensorflow as tf 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
def prepare_data(file_name):
    df = pd.read_csv(r'{}'.format(file_name), names=["path", "target"])
    print(df["path"].head())
    np.where(df.applymap(lambda x: x == ''))
    
    # X Input Data
    image_list = []

    for path in df['path'] :
#         print(path) 
        image_arr = cv2.imread(path)
        image_224x224 = cv2.resize(image_arr, (150, 150))
        image_list.append(image_224x224.astype('float')) 
    X = np.array(image_list)
    print("Total Data = ", X.shape)

    # Y Output Data
    encoder = LabelEncoder()
    Targets = encoder.fit_transform(df['target'])
    n_classes = 2
    Y = to_categorical(Targets, num_classes = n_classes)
    
    return X, Y

**Dataset Hand**

In [6]:
objek = 'hand'

In [6]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_HAND/patient10169/study1_ne...
1    MURA-v1.1/train/XR_HAND/patient10121/study1_po...
2    MURA-v1.1/train/XR_HAND/patient09111/study1_ne...
3    MURA-v1.1/train/XR_HAND/patient11008/study1_ne...
4    MURA-v1.1/train/XR_HAND/patient09780/study1_po...
Name: path, dtype: object
Total Data =  (5543, 150, 150, 3)
Data Train (4711, 150, 150, 3)
Data Val (832, 150, 150, 3)


In [4]:
vgg_model = VGG16(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

for layer in vgg_model.layers:
    layer.trainable = False

x = layers.Flatten()(vgg_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(vgg_model.input, x)

model.compile(optimizer = Adamax(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [8]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
148/148 [==============================] - 524s 4s/step - loss: 0.5841 - acc: 0.7281 - val_loss: 0.5581 - val_acc: 0.7428
Epoch 2/20
148/148 [==============================] - 495s 3s/step - loss: 0.5634 - acc: 0.7304 - val_loss: 0.5533 - val_acc: 0.7428
Epoch 3/20
148/148 [==============================] - 512s 3s/step - loss: 0.5575 - acc: 0.7313 - val_loss: 0.5538 - val_acc: 0.7488
Epoch 4/20
148/148 [==============================] - 466s 3s/step - loss: 0.5474 - acc: 0.7334 - val_loss: 0.5488 - val_acc: 0.7464
Epoch 5/20
148/148 [==============================] - 464s 3s/step - loss: 0.5416 - acc: 0.7349 - val_loss: 0.5465 - val_acc: 0.7488
Epoch 6/20
148/148 [==============================] - 502s 3s/step - loss: 0.5360 - acc: 0.7395 - val_loss: 0.5448 - val_acc: 0.7488
Epoch 7/20
148/148 [==============================] - 502s 3s/step - loss: 0.5329 - acc: 0.7453 - val_loss: 0.5425 - val_acc: 0.7476
Epoch 8/20
148/148 [==============================] - 482s 3s/step - 

In [7]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))

0    MURA-v1.1/valid/XR_HAND/patient11497/study1_po...
1    MURA-v1.1/valid/XR_HAND/patient11497/study1_po...
2    MURA-v1.1/valid/XR_HAND/patient11497/study1_po...
3    MURA-v1.1/valid/XR_HAND/patient11498/study1_po...
4    MURA-v1.1/valid/XR_HAND/patient11498/study1_po...
Name: path, dtype: object
Total Data =  (460, 150, 150, 3)


In [8]:
model_load = load_model("Inception_Model/inception_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

Accuracy over the test set: 
  58.91 %


**Dataset Humerus**

In [11]:
objek = 'humerus'

In [4]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_HUMERUS/patient02717/study1...
1    MURA-v1.1/train/XR_HUMERUS/patient02922/study1...
2    MURA-v1.1/train/XR_HUMERUS/patient02746/study1...
3    MURA-v1.1/train/XR_HUMERUS/patient02917/study1...
4    MURA-v1.1/train/XR_HUMERUS/patient02739/study1...
Name: path, dtype: object
Total Data =  (1272, 150, 150, 3)
Data Train (1081, 150, 150, 3)
Data Val (191, 150, 150, 3)


In [7]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
34/34 [==============================] - 112s 3s/step - loss: 0.7036 - acc: 0.5523 - val_loss: 0.6698 - val_acc: 0.5759
Epoch 2/20
34/34 [==============================] - 121s 4s/step - loss: 0.6645 - acc: 0.6068 - val_loss: 0.6589 - val_acc: 0.6126
Epoch 3/20
34/34 [==============================] - 125s 4s/step - loss: 0.6468 - acc: 0.6457 - val_loss: 0.6403 - val_acc: 0.6597
Epoch 4/20
34/34 [==============================] - 126s 4s/step - loss: 0.6312 - acc: 0.6503 - val_loss: 0.6323 - val_acc: 0.6911
Epoch 5/20
34/34 [==============================] - 126s 4s/step - loss: 0.6171 - acc: 0.6660 - val_loss: 0.6401 - val_acc: 0.6440
Epoch 6/20
34/34 [==============================] - 105s 3s/step - loss: 0.5978 - acc: 0.6994 - val_loss: 0.6313 - val_acc: 0.6754
Epoch 7/20
34/34 [==============================] - 107s 3s/step - loss: 0.5858 - acc: 0.7151 - val_loss: 0.6181 - val_acc: 0.7016
Epoch 8/20
34/34 [==============================] - 107s 3s/step - loss: 0.5811 - a

In [12]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))

0    MURA-v1.1/valid/XR_HUMERUS/patient11608/study1...
1    MURA-v1.1/valid/XR_HUMERUS/patient11608/study1...
2    MURA-v1.1/valid/XR_HUMERUS/patient11214/study1...
3    MURA-v1.1/valid/XR_HUMERUS/patient11214/study1...
4    MURA-v1.1/valid/XR_HUMERUS/patient11609/study1...
Name: path, dtype: object
Total Data =  (288, 150, 150, 3)


In [13]:
model_load = load_model("Inception_Model/inception_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

Accuracy over the test set: 
  50.0 %


**Shoulder Dataset**

In [14]:
objek = 'shoulder'

In [9]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_SHOULDER/patient01685/study...
1    MURA-v1.1/train/XR_SHOULDER/patient02206/study...
2    MURA-v1.1/train/XR_SHOULDER/patient01976/study...
3    MURA-v1.1/train/XR_SHOULDER/patient01071/study...
4    MURA-v1.1/train/XR_SHOULDER/patient02526/study...
Name: path, dtype: object
Total Data =  (8364, 150, 150, 3)
Data Train (7109, 150, 150, 3)
Data Val (1255, 150, 150, 3)


In [10]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
223/223 [==============================] - 803s 4s/step - loss: 0.6841 - acc: 0.5624 - val_loss: 0.6739 - val_acc: 0.5849
Epoch 2/20
223/223 [==============================] - 738s 3s/step - loss: 0.6536 - acc: 0.6140 - val_loss: 0.6604 - val_acc: 0.6096
Epoch 3/20
223/223 [==============================] - 754s 3s/step - loss: 0.6409 - acc: 0.6390 - val_loss: 0.6503 - val_acc: 0.6311
Epoch 4/20
223/223 [==============================] - 788s 4s/step - loss: 0.6303 - acc: 0.6500 - val_loss: 0.6436 - val_acc: 0.6390
Epoch 5/20
223/223 [==============================] - 696s 3s/step - loss: 0.6191 - acc: 0.6662 - val_loss: 0.6406 - val_acc: 0.6375
Epoch 6/20
223/223 [==============================] - 694s 3s/step - loss: 0.6121 - acc: 0.6731 - val_loss: 0.6376 - val_acc: 0.6462
Epoch 7/20
223/223 [==============================] - 698s 3s/step - loss: 0.6055 - acc: 0.6838 - val_loss: 0.6317 - val_acc: 0.6590
Epoch 8/20
223/223 [==============================] - 745s 3s/step - 

In [15]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))

0    MURA-v1.1/valid/XR_SHOULDER/patient11676/study...
1    MURA-v1.1/valid/XR_SHOULDER/patient11676/study...
2    MURA-v1.1/valid/XR_SHOULDER/patient11676/study...
3    MURA-v1.1/valid/XR_SHOULDER/patient11703/study...
4    MURA-v1.1/valid/XR_SHOULDER/patient11703/study...
Name: path, dtype: object
Total Data =  (563, 150, 150, 3)


In [16]:
model_load = load_model("Inception_Model/inception_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

Accuracy over the test set: 
  50.44 %


**Wrist Dataset**

In [17]:
objek = 'wrist'

In [5]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_WRIST/patient07688/study1_n...
1    MURA-v1.1/train/XR_WRIST/patient06904/study1_p...
2    MURA-v1.1/train/XR_WRIST/patient06806/study1_p...
3    MURA-v1.1/train/XR_WRIST/patient06442/study1_p...
4    MURA-v1.1/train/XR_WRIST/patient00305/study1_n...
Name: path, dtype: object
Total Data =  (9739, 150, 150, 3)
Data Train (8278, 150, 150, 3)
Data Val (1461, 150, 150, 3)


In [6]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
259/259 [==============================] - 2391s 9s/step - loss: 0.6337 - acc: 0.6499 - val_loss: 0.5917 - val_acc: 0.6940
Epoch 2/20
259/259 [==============================] - 1517s 6s/step - loss: 0.5767 - acc: 0.6982 - val_loss: 0.5633 - val_acc: 0.7050
Epoch 3/20
259/259 [==============================] - 1378s 5s/step - loss: 0.5506 - acc: 0.7202 - val_loss: 0.5450 - val_acc: 0.7358
Epoch 4/20
259/259 [==============================] - 842s 3s/step - loss: 0.5314 - acc: 0.7354 - val_loss: 0.5328 - val_acc: 0.7509
Epoch 5/20
259/259 [==============================] - 843s 3s/step - loss: 0.5194 - acc: 0.7418 - val_loss: 0.5235 - val_acc: 0.7413
Epoch 6/20
259/259 [==============================] - 804s 3s/step - loss: 0.5078 - acc: 0.7547 - val_loss: 0.5151 - val_acc: 0.7509
Epoch 7/20
259/259 [==============================] - 805s 3s/step - loss: 0.4996 - acc: 0.7597 - val_loss: 0.5101 - val_acc: 0.7563
Epoch 8/20
259/259 [==============================] - 816s 3s/step

In [18]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))

0    MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
1    MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
2    MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
3    MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
4    MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...
Name: path, dtype: object
Total Data =  (659, 150, 150, 3)


In [19]:
model_load = load_model("Inception_Model/inception_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

Accuracy over the test set: 
  55.24 %


**Elbow Dataset**

In [20]:
objek = 'elbow'

In [5]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_ELBOW/patient05456/study1_p...
1    MURA-v1.1/train/XR_ELBOW/patient04779/study1_p...
2    MURA-v1.1/train/XR_ELBOW/patient00222/study1_n...
3    MURA-v1.1/train/XR_ELBOW/patient05301/study1_p...
4    MURA-v1.1/train/XR_ELBOW/patient03193/study1_n...
Name: path, dtype: object
Total Data =  (4931, 150, 150, 3)
Data Train (4191, 150, 150, 3)
Data Val (740, 150, 150, 3)


In [6]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
131/131 [==============================] - 448s 3s/step - loss: 0.6816 - acc: 0.5944 - val_loss: 0.6351 - val_acc: 0.6662
Epoch 2/20
131/131 [==============================] - 435s 3s/step - loss: 0.6406 - acc: 0.6495 - val_loss: 0.6138 - val_acc: 0.6500
Epoch 3/20
131/131 [==============================] - 440s 3s/step - loss: 0.6226 - acc: 0.6698 - val_loss: 0.5970 - val_acc: 0.6716
Epoch 4/20
131/131 [==============================] - 431s 3s/step - loss: 0.6046 - acc: 0.6872 - val_loss: 0.5830 - val_acc: 0.7324
Epoch 5/20
131/131 [==============================] - 426s 3s/step - loss: 0.5916 - acc: 0.7079 - val_loss: 0.5695 - val_acc: 0.7095
Epoch 6/20
131/131 [==============================] - 406s 3s/step - loss: 0.5787 - acc: 0.7132 - val_loss: 0.5604 - val_acc: 0.7203
Epoch 7/20
131/131 [==============================] - 431s 3s/step - loss: 0.5727 - acc: 0.7184 - val_loss: 0.5545 - val_acc: 0.7432
Epoch 8/20
131/131 [==============================] - 436s 3s/step - 

In [21]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))

0    MURA-v1.1/valid/XR_ELBOW/patient11659/study1_p...
1    MURA-v1.1/valid/XR_ELBOW/patient11659/study1_p...
2    MURA-v1.1/valid/XR_ELBOW/patient11659/study1_p...
3    MURA-v1.1/valid/XR_ELBOW/patient11802/study1_p...
4    MURA-v1.1/valid/XR_ELBOW/patient11802/study1_p...
Name: path, dtype: object
Total Data =  (465, 150, 150, 3)


In [22]:
model_load = load_model("Inception_Model/inception_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

Accuracy over the test set: 
  53.33 %


**Finger Dataset**

In [23]:
objek = 'finger'

In [24]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_FINGER/patient03402/study1_...
1    MURA-v1.1/train/XR_FINGER/patient04428/study1_...
2    MURA-v1.1/train/XR_FINGER/patient03833/study1_...
3    MURA-v1.1/train/XR_FINGER/patient04139/study1_...
4    MURA-v1.1/train/XR_FINGER/patient02922/study1_...
Name: path, dtype: object
Total Data =  (5106, 150, 150, 3)
Data Train (4340, 150, 150, 3)
Data Val (766, 150, 150, 3)


In [25]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
136/136 [==============================] - 477s 3s/step - loss: 0.6142 - acc: 0.6578 - val_loss: 0.5863 - val_acc: 0.6880
Epoch 2/20
136/136 [==============================] - 479s 4s/step - loss: 0.5690 - acc: 0.6998 - val_loss: 0.5713 - val_acc: 0.6919
Epoch 3/20
136/136 [==============================] - 458s 3s/step - loss: 0.5523 - acc: 0.7127 - val_loss: 0.5620 - val_acc: 0.6906
Epoch 4/20
136/136 [==============================] - 453s 3s/step - loss: 0.5417 - acc: 0.7062 - val_loss: 0.5651 - val_acc: 0.6854
Epoch 5/20
136/136 [==============================] - 450s 3s/step - loss: 0.5340 - acc: 0.7164 - val_loss: 0.5852 - val_acc: 0.6697
Epoch 6/20
136/136 [==============================] - 466s 3s/step - loss: 0.5233 - acc: 0.7362 - val_loss: 0.5533 - val_acc: 0.6971
Epoch 7/20
136/136 [==============================] - 453s 3s/step - loss: 0.5171 - acc: 0.7359 - val_loss: 0.5476 - val_acc: 0.6997
Epoch 8/20
136/136 [==============================] - 475s 3s/step - 

**Forearm Dataset**

In [26]:
objek = 'forearm'

In [27]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_FOREARM/patient09614/study1...
1    MURA-v1.1/train/XR_FOREARM/patient09541/study1...
2    MURA-v1.1/train/XR_FOREARM/patient01951/study1...
3    MURA-v1.1/train/XR_FOREARM/patient09113/study1...
4    MURA-v1.1/train/XR_FOREARM/patient09552/study1...
Name: path, dtype: object
Total Data =  (1825, 150, 150, 3)
Data Train (1551, 150, 150, 3)
Data Val (274, 150, 150, 3)


In [28]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20)

Epoch 1/20
49/49 [==============================] - 179s 4s/step - loss: 0.6522 - acc: 0.6402 - val_loss: 0.6561 - val_acc: 0.6022
Epoch 2/20
49/49 [==============================] - 156s 3s/step - loss: 0.6101 - acc: 0.6557 - val_loss: 0.6265 - val_acc: 0.6314
Epoch 3/20
49/49 [==============================] - 155s 3s/step - loss: 0.5947 - acc: 0.6828 - val_loss: 0.6301 - val_acc: 0.6496
Epoch 4/20
49/49 [==============================] - 154s 3s/step - loss: 0.5837 - acc: 0.7002 - val_loss: 0.6198 - val_acc: 0.6679
Epoch 5/20
49/49 [==============================] - 158s 3s/step - loss: 0.5707 - acc: 0.7054 - val_loss: 0.6186 - val_acc: 0.6679
Epoch 6/20
49/49 [==============================] - 158s 3s/step - loss: 0.5671 - acc: 0.7157 - val_loss: 0.6158 - val_acc: 0.6679
Epoch 7/20
49/49 [==============================] - 165s 3s/step - loss: 0.5535 - acc: 0.7286 - val_loss: 0.6042 - val_acc: 0.6825
Epoch 8/20
49/49 [==============================] - 157s 3s/step - loss: 0.5513 - a